<a href="https://colab.research.google.com/github/ZacharyZekaiXu/ZekaiXu_CrossmodalRecognition/blob/main/DDQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>DDQN </h1>


In [1]:
!conda --version

conda 4.14.0


In [8]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/skoch9/meshplot.git (to revision 725e4a7926a5f10888f0edd1762fecf9db751c56) to /tmp/pip-install-ieg6m3e0/meshplot_fe1a9db0d0514e2f95453703a1e265da
  Running command git clone --filter=blob:none --quiet https://github.com/skoch9/meshplot.git /tmp/pip-install-ieg6m3e0/meshplot_fe1a9db0d0514e2f95453703a1e265da
  Running command git rev-parse -q --verify 'sha^725e4a7926a5f10888f0edd1762fecf9db751c56'
  Running command git fetch -q https://github.com/skoch9/meshplot.git 725e4a7926a5f10888f0edd1762fecf9db751c56
  Resolved https://github.com/skoch9/meshplot.git to commit 725e4a7926a5f10888f0edd1762fecf9db751c56
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 97.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 8

In [9]:
!bash download_models.sh

--2022-09-05 19:48:27--  https://dl.fbaipublicfiles.com/ABC/ActivePretrained.tar
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 353586521 (337M) [application/gzip]
Saving to: ‘ActivePretrained.tar’

ActivePretrained.ta 100%[===================>] 337.21M  22.8MB/s    in 17s     

2022-09-05 19:48:45 (20.2 MB/s) - ‘ActivePretrained.tar’ saved [353586521/353586521]



In [19]:
!bash download_models.sh

--2022-09-05 20:04:27--  https://dl.fbaipublicfiles.com/ABC/ActivePretrained.tar
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 353586521 (337M) [application/gzip]
Saving to: ‘ActivePretrained.tar’

ActivePretrained.ta 100%[===================>] 337.21M  15.8MB/s    in 24s     

2022-09-05 20:04:52 (13.9 MB/s) - ‘ActivePretrained.tar’ saved [353586521/353586521]



In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:27
🔁 Restarting kernel...


In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
%cd gdrive/MyDrive/Active-3D-Vision-and-Touch/

/content/gdrive/MyDrive/Active-3D-Vision-and-Touch


<h2> 1. DDQN - Training  </h2>

In [10]:
!pip install submitit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
import os
import sys
import torch
need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith("1.12.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.3/596.3 kB 21.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20220512-py3-none-any.whl size=61263 sha256=9ef2378301425ec75ec8e789e863e418fcc9d7621bd6d5b7686bf89b43be20da
  Stored in directory: /root/.cache/pip/wheels/68/20/f9/a11a0dd63f4c13678b2a5ec488e48078756505c7777b75b29e
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31532 sha256=40a53ea17628ee63c8efe1738d7b5d02c744e5710e2ed7987ec9d298e149103e
  Stored in directory: /root/.cache/pip/wheels/aa/cc/ed/ca4e88beef656b01c84b9185196513ef2faf74a5a379b043a7
  Created

In [12]:
!pip install torch
import torch
!pip install trimesh
!pip install pyrender

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 2.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Here we train a DDQN model in the touch only setting with grasping. This model leverages the latent space to learn actions.

In [13]:
!pip3 install pybullet --upgrade
!pip3 install stable_baselines --upgrade
!pip3 install tensorflow==1.14.0 #tensorflow deprecates and breaks their API
#see PyBullet Quickstart Guide at
#https://docs.google.com/document/d/10sXEhzFRSnvFcl3XxNGhnD4N2SedqwdAvK3dsihxVUA/edit
import imageio
import numpy as np



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.0/307.0 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 734.5/734.5 kB 49.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 77.7 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.25.2-py3-none-any.whl size=852302 sha256=2b9c2a898eced27c01d7fe94b99e28c98c94c136f7680c22f87c621a90e65999
  Stored in directory: /root/.cache/p

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.3/109.3 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 81.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.2/75.2 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.5/488.5 kB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 96.7 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.10.0
    Uninstalling tensorboard-2.10.0:
      Successfully uninstalled tensorboard-2.10.0


In [14]:

!pip uninstall tensorboard
!pip uninstall tensorflow
!pip uninstall tensorboard
!pip uninstall tensorboardx
!pip uninstall tensorboard-plugin-wit
!pip uninstall cloud-tpu-client

!pip install tensorboard


Found existing installation: tensorboard 1.14.0
Uninstalling tensorboard-1.14.0:
  Would remove:
    /usr/local/bin/tensorboard
    /usr/local/lib/python3.7/site-packages/tensorboard-1.14.0.dist-info/*
    /usr/local/lib/python3.7/site-packages/tensorboard/*
Proceed (Y/n)? y
  Successfully uninstalled tensorboard-1.14.0
Found existing installation: tensorflow 1.14.0
Uninstalling tensorflow-1.14.0:
  Would remove:
    /usr/local/bin/freeze_graph
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/site-packages/tensorflow-1.14.0.dist-info/*
    /usr/local/lib/python3.7/site-packages/tensorflow/*
Proceed (Y/n)? y
  Successfully uninstalled tensorflow-1.14.0
Found existing installation: tensorboard-plugin-wit 1.8.1
Uninstalling tensorboard-plugin-wit-1.8.1:
  Would remove:
    /usr/local/lib/python3.7/site-packages/tensorboard_plugin_wit-1.8.1.dist-info

In [15]:

from tensorboard import version
print(version.VERSION)

2.10.0


In [16]:

import os

from pterotactyl.policies.DDQN import train as ddqn_train 


class Params: # define training arguments 
    def __init__(self):
        self.cut = 0.33
        self.seed = 0 
        self.gamma = .99
        self.limit_data = True
        self.epochs = 30
        self.lr = 0.0003
        self.eval = False
        self.env_batch_size = 2
        self.num_actions = 50
        self.budget = 5
        self.train_batch_size = 16
        self.epsilon_start = 1.
        self.epsilon_end = 0.001
        self.epsilon_decay = 0.9999
        self.number_points = 10000
        self.loss_coeff = 9000
        self.train_steps = 20
        self.valid_steps = 20 
        self.target_update = 1000
        self.burn_in = 100
        self.use_latent = True
        self.use_recon = False 
        self.exp_id = "example"
        self.exp_type = "ddqn"
        self.finger = False 
        self.num_grasps = 5 
        self.mem_capacity = 1000
        self.hidden_dim = 100
        self.layers = 5
        self.use_touch = True # using touch 
        self.use_img = False # not using vision 
        self.normalization = 'first'
        self.visualize = False
        self.pretrained_recon = True
        self.pretrained=False
    

params = Params()
ddqn_trainer = ddqn_train.Engine(params)


In [17]:
ddqn_train.Engine(params)

In [20]:
ddqn_trainer()


/usr/local/lib/python3.7/site-packages/pytorch3d/io/obj_io.py:529: UserWarning: No mtl file provided
  warnings.warn("No mtl file provided")


FileNotFoundError: ignored

<h2> 2. DDQN - Evaluation </h2>

Here we evaluate the trained DDQN model in the touch only grasping setting. 

In [17]:
import os
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw 
from glob import glob 

from pterotactyl import objects
FONT_LOCATION = os.path.dirname(objects.__file__) + '/OpenSans-Bold.ttf'


params.eval = True 
params.visualize = True
tester = ddqn_train.Engine(params)
tester()



histogram = Image.open('results/ddqn/example/histogram.png')
display(histogram)
print('\n\n')
actions = Image.open('results/ddqn/example/sphere_projection.png')
print('   Projected Actions')
display(actions)


img_locations = glob('results/ddqn/example/*/')
for location in img_locations[:5]:
    mesh = Image.open(location + "mesh.png")
    points = Image.open(location + "points.png")
    gt = Image.open(location + "ground_truth.png")
    
    
    combo = Image.new('RGB', (512*3, 512))
    x_offset = 0
    
    for im, st in [[gt, 'Ground Truth'], [mesh, 'Mesh'], [points, 'Point Cloud']]:
        
        draw = ImageDraw.Draw(im)
        font = ImageFont.truetype(FONT_LOCATION, 35)
        W,H = im.size
        w,h = font.getsize(st)
        draw.text(((W-w)/2, 0),st,(0, 0, 0), font = font)
        combo.paste(im, (x_offset, 0))
        x_offset += im.size[0]
        
    display(combo)








FileNotFoundError: ignored

<h2> DDQN - Pretrained </h2>

We now evaluate the pretrained policy: 

In [ ]:
import os

from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw 
from glob import glob 

from pterotactyl import objects
from pterotactyl.policies.DDQN import train as ddqn_train 

FONT_LOCATION = os.path.dirname(objects.__file__) + '/OpenSans-Bold.ttf'


class Params: # define training arguments 
    def __init__(self):
        self.cut = 0.33
        self.seed = 0 
        self.gamma = .99
        self.limit_data = True
        self.epochs = 30
        self.lr = 0.0003
        self.eval = True
        self.env_batch_size = 2
        self.num_actions = 50
        self.budget = 5
        self.train_batch_size = 16
        self.epsilon_start = 1.
        self.epsilon_end = 0.001
        self.epsilon_decay = 0.9999
        self.number_points = 10000
        self.loss_coeff = 9000
        self.train_steps = 20
        self.valid_steps = 20 
        self.target_update = 1000
        self.burn_in = 100
        self.use_latent = False
        self.use_recon = True 
        self.exp_id = "ddqn_pre"
        self.exp_type = "ddqn"
        self.finger = False 
        self.num_grasps = 5 
        self.mem_capacity = 1000
        self.hidden_dim = 100
        self.layers = 5
        self.use_touch = True # using touch 
        self.use_img = False # not using vision 
        self.normalization = 'first'
        self.visualize = True
        self.pretrained_recon = True
        self.pretrained = True 
    

params = Params()
eval = ddqn_train.Engine(params)
eval()


histogram = Image.open('results/ddqn/ddqn_pre/histogram.png')
display(histogram)
print('\n\n')
actions = Image.open('results/ddqn/ddqn_pre/sphere_projection.png')
print('   Projected Actions')
display(actions)



img_locations = glob('results/ddqn/ddqn_pre/*/')
for location in img_locations[:5]:
    mesh = Image.open(location + "mesh.png")
    points = Image.open(location + "points.png")
    gt = Image.open(location + "ground_truth.png")
    
    
    combo = Image.new('RGB', (512*3, 512))
    x_offset = 0
    
    for im, st in [[gt, 'Ground Truth'], [mesh, 'Mesh'], [points, 'Point Cloud']]:
        
        draw = ImageDraw.Draw(im)
        font = ImageFont.truetype(FONT_LOCATION, 35)
        W,H = im.size
        w,h = font.getsize(st)
        draw.text(((W-w)/2, 0),st,(0, 0, 0), font = font)
        combo.paste(im, (x_offset, 0))
        x_offset += im.size[0]
        
    display(combo)






